In [1]:
import random
import os
import glob
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2

from tqdm import tqdm

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D,MaxPool2D,Dense,Dropout,Flatten,BatchNormalization,AvgPool2D
from keras.models import Sequential,save_model,load_model
from keras.optimizers import Adam

from keras.utils.np_utils import to_categorical

from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau


from sklearn.metrics import confusion_matrix,classification_report


In [2]:
path = "C:\\Users\\Intel\\Documents\\Coronahack-Chest-XRay-Dataset\\Coronahack-Chest-XRay-Dataset"

In [3]:
train_dir = os.path.join(path,"train")
test_dir = os.path.join(path,"test")



In [4]:
print("Total num of train Images are: ",str(len(os.listdir(train_dir))))
print()
print("Total num of test Images are: ",str(len(os.listdir(test_dir))))

Total num of train Images are:  5309

Total num of test Images are:  624


In [5]:
BATCH_SIZE = 50
IMG_SHAPE = 256

In [6]:
def get_label(img):
    name = img.split("-")[0]
    if name == "NORMAL2": return 0
    elif name == "IM" : return 0
    else: return 1

In [7]:
def creat_train_data():
    train = []
    
    for img in tqdm(os.listdir(train_dir)):
        label = get_label(img)
        img_path = os.path.join(train_dir,img)
        image = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image,(256,256))
        image = np.array(image).reshape(-1,IMG_SHAPE,IMG_SHAPE,1)
        train.append([np.array(image),np.array(label)])
    random.shuffle(train)
    
    
    return train

In [8]:
train_dataset = creat_train_data()

100%|██████████| 5309/5309 [04:04<00:00, 21.75it/s] 


In [9]:
def creat_test_data():
    test = []
    for img in tqdm(os.listdir(test_dir)):
        label = get_label(img)
        img_path = os.path.join(test_dir,img)
        image = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image,(256,256))
        image = np.array(image).reshape(-1,IMG_SHAPE,IMG_SHAPE,1)
        test.append([np.array(image),np.array(label)])
    random.shuffle(test)
    np.save("test_data.npy",test)
    
    return test

In [10]:
test_dataset = creat_test_data()


100%|██████████| 624/624 [00:29<00:00, 21.06it/s]
C:\Users\Intel\OneDrive\Attachments\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [11]:
len(test_dataset)

624

In [12]:
X_test = np.array([i[0] for i in test_dataset]).reshape(-1,IMG_SHAPE,IMG_SHAPE,1)
Y_test = np.array([i[1] for i in test_dataset])

In [13]:
print(X_test.shape)
print(Y_test.shape)

(624, 256, 256, 1)
(624,)


In [14]:
X_train = np.array([i[0] for i in train_dataset]).reshape(-1,IMG_SHAPE,IMG_SHAPE,1)
Y_train = np.array([i[1] for i in train_dataset])

In [15]:
print(X_train.shape)
print(Y_train.shape)

(5309, 256, 256, 1)
(5309,)


In [16]:
model = Sequential()

model.add(Conv2D(filters=32,kernel_size=5,activation="relu",input_shape=(IMG_SHAPE,IMG_SHAPE,1)))
model.add(Conv2D(filters=32,kernel_size=5,activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(filters=64,kernel_size=5,activation="relu"))
model.add(Conv2D(filters=64,kernel_size=5,activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Dropout(0.2))

model.add(Conv2D(filters=128,kernel_size=5,activation="relu"))
model.add(Conv2D(filters=128,kernel_size=5,activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(filters=256,kernel_size=5,activation="relu"))
model.add(Conv2D(filters=256,kernel_size=5,activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(filters=1024,kernel_size=5,activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dropout(0.25))

model.add(Dense(units=1024,activation="relu"))
model.add(Dropout(0.5))

model.add(Dense(units=128,activation="relu"))
model.add(Dense(units=2, activation="softmax"))


In [17]:
model.compile(Adam(lr=0.001),loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 252, 252, 32)      832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 248, 248, 32)      25632     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 124, 124, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 124, 124, 32)      128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 120, 120, 64)      51264     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 116, 116, 64)      102464    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 58, 58, 64)        0

In [19]:
checkpoint = ModelCheckpoint(filepath = "/",
    monitor='val_loss',
    verbose=1,
    mode = "min",
    save_best_only=True)

earlystop = EarlyStopping(monitor='val_loss',
    min_delta=0,
    patience=9,
    verbose=1,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
    factor=0.2,
    patience=6,
    verbose=1,
    min_delta=0.0001)


callbacks = [earlystop, reduce_lr]